In [2]:
import torch
import sys 
sys.path.append('..')
from models.yolo import get_model
from config import config
import cv2
import numpy as np
from datasets import VOCDataset
from torch.utils.data import DataLoader
import torchvision

In [3]:
model = get_model(config)
model_dict = torch.load('../log/Epoch_48.pt')['state_dict']
model.load_state_dict(model_dict)

<All keys matched successfully>

In [4]:
def read_image_form_torch(image_path):
    image = cv2.imread(image_path)
    image = image/255.0
    image = np.expand_dims(np.transpose(image,[2,0,1]),0) 
    image = torch.from_numpy(image)
    return image

In [5]:
root = "/mnt/e/AllData/VOC2012"
test_datasets = VOCDataset(True, 640, root)

In [6]:
test_dataloader = DataLoader(test_datasets,batch_size=4,collate_fn=test_datasets.collate_fn)

In [12]:
anchors = torch.tensor([[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]]).view(3, 3, 2)
for image,labels,visual_info in test_dataloader:
    predicts = model(image)
    b,_,_,_ = predicts[0].shape
    objs=[]
    for i,(predict,stride) in enumerate(zip(predicts,[8,16,32])):
        b,c,h,w = predict.shape
        predict = predict.view(b,3,5+20,h,w).permute(0, 1, 3, 4, 2).contiguous()
        predict = predict.view(b, 3, -1, predict.size(-1)).sigmoid()
        layer_confidence = predict[..., [4]] * predict[..., 5:]
        keep_batch_indices, keep_anchor_indices, keep_cell_indices, object_classes = torch.where(layer_confidence>0.3)
        object_score = layer_confidence[keep_batch_indices,keep_anchor_indices,keep_cell_indices,object_classes]

        keepbox = predict[keep_batch_indices, keep_anchor_indices, keep_cell_indices].float()
        layer_anchors = anchors[i]
        keep_anchors = layer_anchors[keep_anchor_indices]
        cell_x = keep_cell_indices % w
        cell_y = keep_cell_indices // w
        keep_cell_xy = torch.cat([cell_x.view(-1, 1), cell_y.view(-1, 1)], dim=1)
        wh_restore = (torch.pow(keepbox[:, 2:4] * 2, 2) * keep_anchors) * stride
        xy_restore = (keepbox[:, :2] * 2.0 - 0.5 + keep_cell_xy) * stride
        object_score = object_score.float().view(-1, 1)
        object_classes = object_classes.float().view(-1, 1)
        keep_batch_indices = keep_batch_indices.float().view(-1, 1)
        box = torch.cat((keep_batch_indices, xy_restore - (wh_restore - 1) * 0.5, xy_restore + (wh_restore - 1) * 0.5, object_score, object_classes), dim=1)
        objs.append(box)
        # if len(torch.where(layer_confidence > 0.1)[0]):
        #     print(torch.where(layer_confidence > 0.1))
    if len(objs) > 0:
        objs_cat = torch.cat(objs, dim=0)
        objs_image_base = []
        for ibatch in range(b):
            # left, top, right, bottom, score, classes
            select_box = objs_cat[objs_cat[:, 0] == ibatch, 1:]
            objs_image_base.append(select_box)
    else:
        objs_image_base = [torch.zeros((0, 6)) for _ in range(b)]
    if 0.5 is not None:
        # 使用类内的nms，类间不做操作
        for ibatch in range(b):
            image_objs = objs_image_base[ibatch]
            if len(image_objs) > 0:
                max_wh_size = 4096
                classes = image_objs[:, [5]]
                bboxes = image_objs[:, :4] + (classes * max_wh_size)
                confidence = image_objs[:, 4]
                keep_index = torchvision.ops.boxes.nms(bboxes, confidence, 0.5)
                objs_image_base[ibatch] = image_objs[keep_index]
    break

In [21]:
objs_image_base[0]

tensor([[2.5599e+02, 2.7079e+02, 5.8655e+02, 6.4272e+02, 4.8398e-01, 1.4000e+01]],
       grad_fn=<IndexBackward>)

In [16]:
import matplotlib.pyplot as plt

In [20]:
visual_info[2]*[visual_info[1].shape[1],visual_info[1].shape[0],visual_info[1].shape[1],visual_info[1].shape[0],1]

array([[272.83000946, 263.63788605, 267.99701691, 192.49513626,
         13.        ],
       [354.64916229, 255.21368027, 207.50047684, 207.36139297,
         14.        ]])

In [25]:
visual_info[0]

3

In [2]:
import sys 
sys.path.append('..')
from models.yolo import get_model

In [38]:
class config:
    model_name  ='yolov5s'
    num_classes = 80

In [50]:
model = get_model(config)

In [51]:
len(model.state_dict().keys())

368

In [52]:
import torch
checkpoint = torch.load('/home/zhanggong/Extern/workspace/yolo_serise/yolov5_self/yolov5s.pt')

In [53]:
weights =[]
for k in checkpoint:
    weights.append(checkpoint[k])

In [54]:
model_weights =[]
model_keys = []
for k in model.state_dict():
    # if 'running_mean' in k :continue
    # if 'running_var' in k:continue
    # if 'num_batches_tracked' in k:continue
    model_weights.append(model.state_dict()[k])
    model_keys.append(k)

In [55]:
len(weights)

368

In [56]:
new_dict = {}
for i,weight in enumerate(weights):
    new_dict[model_keys[i]] =  weight
torch.save(new_dict,'yolov5_coco.pt')


In [57]:
for i in range(len(weights)):
    print(weights[i].shape==model_weights[i].shape)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [35]:
weights[-1].shape

torch.Size([255])

In [37]:
model_weights[-1].shape

torch.Size([75])